In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas  as pd

pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',1000)

import seaborn as sns

In [2]:
df= pd.read_csv('../data/processed/unique.csv')

In [3]:
df.head()

,Brand,Mobile Name,Front Camera (in MP),Primary Camera (in MP),Memory,RAM,Processor Performance,Battery Power(in mAh),Screen Height,Screen Width,Ratings,Price,Mobile Category,Price Range
0,Apple,iPhone 14 Pro Max,32,200,1024,24,5.0,1074.0,2520,1080,4.0,119405.30,Flagship,130000-140000
1,Apple,iPhone 14 Pro Max,32,200,512,24,5.0,1074.0,2520,1080,4.9,113962.24,Flagship,110000-120000
2,Apple,iPhone 14 Pro,32,200,512,24,5.0,1086.0,2460,1080,3.4,222232.56,Flagship,220000-230000
3,Apple,iPhone 14 Pro Max,32,200,1024,16,5.0,1074.0,2520,1080,4.2,219022.20,Flagship,210000-220000
4,Apple,iPhone 12 Pro,32,108,1024,16,5.0,1788.5,2660,1170,4.4,216084.16,Flagship,210000-220000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316 entries, 0 to 1315
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Brand                   1316 non-null   object 
 1   Mobile Name             1316 non-null   object 
 2   Front Camera (in MP)    1316 non-null   int64  
 3   Primary Camera (in MP)  1316 non-null   int64  
 4   Memory                  1316 non-null   int64  
 5   RAM                     1316 non-null   int64  
 6   Processor Performance   1316 non-null   float64
 7   Battery Power(in mAh)   1316 non-null   float64
 8   Screen Height           1316 non-null   int64  
 9   Screen Width            1316 non-null   int64  
 10  Ratings                 1316 non-null   float64
 11  Price                   1316 non-null   float64
 12  Mobile Category         1316 non-null   object 
 13  Price Range             1316 non-null   object 
dtypes: float64(4), int64(6), object(4)
memor

In [5]:
df['Price Range'].unique()

array(['130000-140000', '110000-120000', '220000-230000', '210000-220000',
       '200000-210000', '180000-190000', '170000-180000', '160000-170000',
       '150000-160000', '140000-150000', '120000-130000', '100000-110000',
       '90000-100000', '80000-90000', '70000-80000', '60000-70000',
       '50000-60000', '40000-50000', '30000-40000', '20000-30000',
       '10000-20000', '0-10000'], dtype=object)

In [7]:
df.head()

,Brand,Mobile Name,Front Camera (in MP),Primary Camera (in MP),Memory,RAM,Processor Performance,Battery Power(in mAh),Screen Height,Screen Width,Ratings,Price,Mobile Category,Price Range
0,Apple,iPhone 14 Pro Max,32,200,1024,24,5.0,1074.0,2520,1080,4.0,119405.30,Flagship,130000-140000
1,Apple,iPhone 14 Pro Max,32,200,512,24,5.0,1074.0,2520,1080,4.9,113962.24,Flagship,110000-120000
2,Apple,iPhone 14 Pro,32,200,512,24,5.0,1086.0,2460,1080,3.4,222232.56,Flagship,220000-230000
3,Apple,iPhone 14 Pro Max,32,200,1024,16,5.0,1074.0,2520,1080,4.2,219022.20,Flagship,210000-220000
4,Apple,iPhone 12 Pro,32,108,1024,16,5.0,1788.5,2660,1170,4.4,216084.16,Flagship,210000-220000


In [9]:
df.groupby(by=['Brand']).count()

,Mobile Name,Front Camera (in MP),Primary Camera (in MP),Memory,RAM,Processor Performance,Battery Power(in mAh),Screen Height,Screen Width,Ratings,Price,Mobile Category,Price Range
Brand,,,,,,,,,,,,,
Apple,133,133,133,133,133,133,133,133,133,133,133,133,133
Huawei,41,41,41,41,41,41,41,41,41,41,41,41,41
Motorola,156,156,156,156,156,156,156,156,156,156,156,156,156
Oneplus,115,115,115,115,115,115,115,115,115,115,115,115,115
Oppo,110,110,110,110,110,110,110,110,110,110,110,110,110
Pixel,82,82,82,82,82,82,82,82,82,82,82,82,82
Realme,140,140,140,140,140,140,140,140,140,140,140,140,140
RedMi,174,174,174,174,174,174,174,174,174,174,174,174,174
Samsung,246,246,246,246,246,246,246,246,246,246,246,246,246


In [30]:
# df.to_csv('../data/processed/unique.csv', index=False)

In [61]:
# df.to_csv('../data/processed/statcleaned.csv', index=False)